# Analyse des données NHANES (Étude Diabète, Âge, IMC)

## Introduction
Ce notebook présente une analyse des données du **National Health and Nutrition Examination Survey (NHANES)**, cycle 2017-2018 (J).

L'objectif est d'explorer les relations entre :
- **L'Indice de Masse Corporelle (IMC)**
- **L'Âge**
- **Le Statut Diabétique**

Nous allons suivre un pipeline de données complet : installation des dépendances, chargement des modules NHANES, fusion des tables, nettoyage des données et analyses statistiques (descriptives et inférentielles).

## 1. Installation et Chargement des Bibliothèques
Nous utilisons principalement le package `haven` pour lire les fichiers XPT, `dplyr` pour la manipulation de données, et `janitor` pour le nettoyage des noms de colonnes.

In [1]:
install.packages(c("haven","dplyr","janitor","ggplot2"))
library(haven)
library(dplyr)
library(janitor)
library(ggplot2)

## 2. Chargement des données NHANES
Nous chargeons les quatre modules principaux requis pour l'étude : Démographie, Anthropométrie (IMC), Questionnaire Diabète et Glycémie.

In [2]:
demo <- read_xpt("Data/DEMO_J.xpt") %>% clean_names()
bmx  <- read_xpt("Data/BMX_J.xpt")  %>% clean_names()
diq  <- read_xpt("Data/DIQ_J.xpt")  %>% clean_names()
glu  <- read_xpt("Data/GLU_J.xpt")  %>% clean_names()

## 3. Fusion et Inspection
Nous fusionnons les tables en utilisant l'identifiant unique `seqn`. Ensuite, nous vérifions les dimensions des données.

In [ ]:
df <- demo %>%
  left_join(bmx, by = "seqn") %>%
  left_join(diq, by = "seqn") %>%
  left_join(glu, by = "seqn")

dim(df)

## 4. Nettoyage et Recodage des Variables
Sélection des variables d'intérêt et recodage des valeurs catégorielles (Sexe et Diabète) pour faciliter l'analyse.

In [ ]:
df_clean <- df %>%
  select(
    seqn,
    age = ridageyr,
    sex = riagendr,
    bmi = bmxbmi,
    diabetes = diq010,
    glucose = lbxglu
  ) %>%
  mutate(
    diabetes_bin = case_when(
      diabetes == 1 ~ 1,
      diabetes == 2 ~ 0,
      TRUE ~ NA_real_
    ),
    sex = case_when(
      sex == 1 ~ "Male",
      sex == 2 ~ "Female",
      TRUE ~ NA_character_
    )
  )

head(df_clean)

## 5. Analyses Descriptives
### Profil démographique et métabolique
Exploration de la distribution de l'âge, de l'IMC et du statut diabétique. Notez que la moyenne d'IMC dans cet échantillon est souvent un indicateur de la prévalence du surpoids.

In [ ]:
summary(df_clean$age)
summary(df_clean$bmi)
table(df_clean$diabetes_bin)


## 6. Visualisations Graphiques
### Impact de l'adiposité
Comparaison de la distribution de l'IMC selon le statut diabétique. Le décalage vers la droite (hautes valeurs d'IMC) pour les diabétiques illustre la corrélation entre obésité et dysfonctionnement métabolique.

In [ ]:
ggplot(df_clean, aes(x = bmi, fill = factor(diabetes_bin))) +
  geom_histogram(alpha = 0.6, bins = 30, position = "identity") +
  labs(
    title = "Distribution de l'IMC par statut diabétique",
    x = "Indice de Masse Corporelle (IMC)",
    y = "Effectif",
    fill = "Diabète (1=Oui, 0=Non)"
  ) +
  theme_minimal()

## 7. Modélisation : Régression Logistique Multivariée
### Quantification du risque (Odds Ratios)
Nous testons l'influence combinée de l'âge, de l'IMC et du sexe. L'interprétation des Odds Ratios (OR) permet de quantifier l'augmentation du risque.

In [ ]:
modele_final <- glm(
  diabetes_bin ~ age + bmi + sex,
  data = df_clean,
  family = binomial
)
summary(modele_final)

# Calcul des Odds Ratios pour une interprétation concrète
exp(coef(modele_final))

### Interprétation des résultats du modèle
- **Âge (OR ≈ 1.06)** : Chaque année supplémentaire augmente le risque de diabète d'environ 6%, toutes choses égales par ailleurs.
- **IMC (OR ≈ 1.07)** : Chaque point d'IMC supplémentaire accroît les chances de diabète de 7%, confirmant l'impact critique de la surcharge pondérale.
- **Sexe (OR ≈ 1.48 pour Male)** : Les hommes de cet échantillon ont environ 48% de risques en plus que les femmes d'être diabétiques, après ajustement sur l'âge et l'IMC.

## Conclusion et Perspectives
L'analyse confirme que l'obésité et le vieillissement sont les piliers de l'épidémiologie du diabète dans l'étude NHANES. 

**Limites** : L'auto-déclaration induit un biais potentiel. Une suite pertinente serait d'intégrer la variable `glucose` (glycémie à jeun) pour identifier les diabétiques non diagnostiqués.